<br>
<a href="https://www.nvidia.cn/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b> Notebook  3：</b> 使用 LangGraph 进行智能循环</h2>
<br>

**欢迎来到课程的第三部分！**

我们现在知道可以将一个 LLM 客户端锁定到各种有用的配置中；可以围绕 LLM 流程化或手动工程化接口，以实现路由、检索、软件查询等，能够在不同的抽象层次上应对这些挑战。还简要尝试了 LangGraph，看起来这是 LangChain 的智能支持机制推荐的接口。在接下来的部分中，我们将正式讨论 LangGraph 的优缺点，并了解如何利用它创建几乎任意智能系统。

### **学习目标：**
- 什么是 LangGraph，我们为什么要学习如何使用它。
- 如何使用 LangGraph 的抽象实现有趣的智能系统。

<hr><br>

## **第一部分：** 在复杂环境中的推理

回想一下理想的智能体定义：
- 理想的智能体能够合理地将任何输入映射到任何输出，无论复杂性或长度如何。

**之前的 Notebook 已经建立了通过典型的 LLM 创建良好输出本质上并不是一件简单的事，但有一些技术可以改善这个过程：**
- **思维链提示** 可以帮助将智能体引导到正确的方向。
- **算法执行** 可以帮助 LLM 使用定义的算法解决问题。
- **结构化输出** 可以帮助 LLM 对给定的结构进行参数化。

我们现在已经定义了这些能力如何用于帮助系统在工具之间、沿着路径以及朝着理想配置进行路由。在这一部分，我们将研究 LangChain 推荐的实现这种灵活系统的机制，并试图了解它们为什么可能是必需的。

----
#### **题外话：预备知识**

这门课程强烈假设您已经修过系列中的一些前置课程，包括[**《利用提示工程构建大语言模型（LLM）应用》**](https://www.nvidia.cn/training/instructor-led-workshops/building-llm-applications-with-prompt-engineering/)、[**《构建基于大语言模型 (LLM) 的应用》**](https://www.nvidia.cn/training/instructor-led-workshops/rapid-application-development-using-large-language-models/)和[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)。

**如果您修过这些课程，您会知道我们*不需要*一个真正的智能体框架来实现 LLM 指定的控制流。** 所有这些系统都使用某种特定的定制系统来创建适合叙事的智能体，并允许控制流的修改。

接下来的所有场景都是实现 LLM 控制流的完全可行机制：

```python
##########################################################
## Prompt Engineering-Based Fulfillment (in Prompt Eng)
while True:
    prompt = (
        "Chat with the person and be a helpful responding assistant."
        " If they want to stop, output <STOP> at the end of your message.\n\nUSER: " + input("[User]")
    )
    output = llm.invoke(prompt)
    if output.content.strip().endswith("<STOP>"):
        break

##########################################################
## Structured-Output Fulfillment (in RAG course)
llm_routes = {
    "logged_in": struct_llm1,
    "logged_out": struct_llm2,
    "reject_path": reject_llm,
}
llm_chain = llm_routes["logged_out"]
while True:
    prompt = "Fill out the person schema. If found in database, you will see a response. {schema}\n\n{input}"
    output_dict = llm_chain({"schema": schema, "input": input("[User]")})
    if output_dict.get("name") and to_db_query(output_dict):
        llm_chain = llm_routes["logged_in"]
    # ...

##########################################################
## Running-State Chain (in RAG Course)
retrieval_chain = (
    RunnablePassthrough() 
    ## {"input", **}
    | RunnablePassthrough.assign({"retrieval": retriever_chain}) 
    ## -> {"input", "retrieval", **}
    | RunnablePassthrough.assign({"requery": rephrase_chain})    
    ## -> {"input", "retrieval", "requery", **}
    | RunnablePassthrough.assign({"response": prompt | llm | StrOutputParser()})
    ## -> {"input", "retrieval", "requery", "response"}
).invoke({"input" : "hello world!"})
```

如果您还没有修过这些课程，也没关系！但我们将直接进入 LangGraph，以避免重复以前的内容。其他课程可以重新查看，并且都可以在自学环境中使用。

> **需要注意的是：这是一个专门为实现智能体而设计的框架！** 它集成了许多复杂性，可能对简单的 LLM 系统没有必要，但我们还是会继续使用它，这样您可以在本课程结束时与最前沿的解决方案进行交互。

<hr><br>

## **第2部分：介绍 LangGraph**

这门课程们将介绍一个新的工具 **[LangGraph](https://github.com/langchain-ai/langgraph)**，它可以通过状态图系统来管理对话流。借助 LangGraph，我们可以以结构化的方式定义智能体的状态、转变和动作，免去完全自定义事件循环的需求。这个框架增强了可扩展性和可维护性，特别是在处理多智能体系统或复杂工作流时。

作为一个框架，它显然有自己的营销材料，传达了许多优秀的功能，因此对感兴趣的人只提供主页的链接。随着课程的进展，您将发现它的价值所在，并自行判断它的优缺点——每个框架都有这些，所有框架都会首先强调优点。

> <a href="https://langchain-ai.github.io/langgraph/" target="_blank"><img src="images/langgraph-intro.png" style="width: 600px" /></a>
> 
> [**LangGraph 首页**](https://www.langchain.com/langgraph)

<br>

#### 为什么 LangGraph 一般来说很棒？

<details>

- **因为它考虑了很多**。LangGraph 可定制性极强，甚至可能迫使您遵循一些最佳实践和限制，这些在您刚开始学习时可能显得毫无意义……但在您真正尝试自定义、扩展和生产化时，就会显得很有影响。
- **因为它已经被广泛采用**。目前有很多示例和现成的解决方案可以让人们直接使用，还有不少研究项目和最终部署都是用它完成的。
- **因为这些技术是可以转移的**。如果您能真正理解 LangGraph，其他框架的优缺点就会变得好理解。如果您只接触过最简单的框架，所有后续的框架就会显得过于复杂。

</details>

#### 那么，LangGraph 比定制方案更好吗？

<details>

- **当您知道自己需要抽象时：** 不同于我们可以转化为可行多状态系统的 while 循环，LangGraph 采用状态图的方法来建模智能体遍历过程。因此，它结合了自然适用于非顺序甚至动态流程的设计模式。
- **当您不知道从何开始，但知道想要生产化时：** 不同于我们可以转化为可行多状态系统的 while 循环，LangGraph 采用状态图的方法来建模智能体遍历过程。因此，它结合了自然适用于非顺序甚至动态流程的设计模式。

</details>

#### LangGraph 什么时候比定制方案差？

<details>

- **当您只是想用一些 LLM 和提示词制作一个简单应用时：** 为了考虑各种特定于多智能体的功能集和边界情况，LangGraph 实现了一些强假设，这大大增加了学习曲线。如果您可以在基本 LangChain 中实现您的解决方案，运行时范式足以简化您的工作流，并绕过 LangGraph 引入的几层复杂性。反之，如果您知道自己想扩展应用，且能从其精心设计的功能/示例中获益，那么深入探索并熟悉它可能是很值得的。
- **当您需要最深入的优化时：** 虽然 LangGraph 很棒，但在 LangGraph 之外仍然有更深层次的优化和更强的模块化。寻找高度专业化微服务的人可能会对自定义多线程/多进程方案、高级图算法和 LangGraph 可能无法提供的高级资源管理战略感兴趣。
- **当您只想要一些个性化智能体时：** 您见过 CrewAI API。那个有相对较平坦的学习曲线，但也没有那么可定制，而且很针对特定用例。如果您想要一种更轻便的方式，那是个相对简单的切入点（但也便于后续回溯）。

</details>

----

<br>

## **第3部分：适应 LangGraph 抽象**

**LangGraph** 基于 LangChain 的低级运行时（LCEL）之上，但引入了更强的**状态**和**转换**的概念。在核心部分，您需要定义：

1. **状态（State）**（在 Python 术语中是一个类型化字典），用于捕捉应用所需的相关信息。  
2. **节点（Nodes）**，每个节点都是一个 Python 函数，*读取*和*更新*该状态。  
3. **边（Edges）**，将这些节点链接在一个有向图中——指明控制如何从一个节点移动到另一个节点。

这不仅仅是“构建工作流”。通过描述节点和边，您实际上在描述智能体如何在可能性环境中进行遍历。

#### **例子：** 一个简单的 2 节点图

下面是使用此框架实现的一个紧凑的应用：

In [ ]:
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

In [ ]:
import uuid
from typing import Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from functools import partial

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    foo: str
    human_value: Optional[str]
    llm_response: Optional[str]
    extra_kwargs: Optional[dict]

##################################################################
## Define the operations (Nodes) that can happen on your environment

def edged_input(state: State):
    """Edge" option where transition is generated at runtime"""
    answer = interrupt("[User]: ")
    print(f"> Received an input from the interrupt: {answer}")
    return {"human_value": answer}           ## <- Edge determined by graph construction (default END)
    # return Command(update={"human_value": answer}, goto="response")

def response(state: State, config=None):
    ## Passing in config will cause connector to stream values to state modification buffer. See graph stream later
    response = llm.invoke(state.get("human_value"), config=config)
    return {"llm_response": response}
    # return Command(update={"llm_response": response}, goto=END)

##################################################################
## Define the system that organizes your nodes (and maybe edges)

builder = StateGraph(State)
builder.add_edge(START, "input")  ## A start node is always necessary
builder.add_node("response", response)
builder.add_node("input", edged_input)
builder.add_edge("input", "response")

##################################################################
## A memory management system to keep track of agent state
checkpointer = MemorySaver()
app = builder.compile(checkpointer=checkpointer)

##################################################################
## A config to define which state you want to use (i.e. states["thread_id"] will be active state pool)
config = {
    "configurable": {
        "thread_id": uuid.uuid4(),
    }
}

app_stream = partial(app.stream, config=config)

#### 分解流程

1. **开始** → **输入**：  
   - 系统从内置的 `START` 占位符跳转到名为 `"input"` 的节点。  
   - `"input"` 绑定到函数 `edged_input()`，该函数*中断*以请求用户输入。用户输入被存储为 `state["human_value"]`。
       - 中断字面上切断了控制流，图的执行在暂停状态结束。 

3. **输入** → **响应**：接着，我们转到 `"response"`，调用 `response()` 函数。

    - 它获取 `state["human_value"]`（用户的文本）并将其传递给 `llm.invoke(...)`。  
    - 得到的 LLM 输出存储在 `state["llm_response"]` 中。

5. **响应** → **结束**：然后，我们将进入最后一个状态。
   - 我们没有告诉 LangGraph 继续往其他地方。没有从 `"response"` 到另一个节点（或再次返回 `"node"`）的额外边，它将直接结束。换句话说，控制流会保持在休眠状态。
   - 您可以添加更多的边以增加更多步骤，或一个返回 `"input"` 的循环以进行多轮交互。

#### 实际运行

有很多方法可以运行这个新链条，表面看起来可能与我们之前玩过的运行时有些相似。不过，您会注意到中断机制和状态缓冲区需要更多的处理。需要将这个系统与其他项目深度集成的软件工程师会对此感到高兴，但刚接触的人可能会觉得有点迷茫。

In [ ]:
##################################################################
## Simple Invocation Example

## We can stream over it until an interrupt is received
for chunk in app_stream({"foo": "abc"}):
    print("Chunk Before Command:", repr(chunk))

## If an interrupt is recieved, we can resolve it and continue
if "__interrupt__" in chunk:
    command = Command(resume=input(chunk.get("__interrupt__")[0].value))

##################################################################
## Follow-Up Example (Without streaming is simple case, with streaming is advanced case)

stream_outputs = True

if not stream_outputs:
    ## Stream just the individual outputs from the state-writing buffer
    print("\nSending Command:", repr(command), "\n")
    for chunk in app_stream(command):
        print("\nChunk After Command:", repr(chunk))

else:
    ## Same thing, but actually populate the message stream with streamed responses auto-magically
    seen_metas = dict()
    for chunk, meta in app_stream(command, stream_mode="messages"):
        if meta.get("checkpoint_ns") not in seen_metas:
            print(f"[{meta.get('langgraph_node')}]: ", end="", flush=True)
            # print(f"\nNew Buffer Stream Meta: {meta}\n")
        seen_metas[meta.get("checkpoint_ns")] = meta
        if chunk.content:
            print(chunk.content, end="", flush=True)
        if chunk.response_metadata: # or chunk.usage_metadata: 
            print(f"\n\nChunk with response_metadata: {repr(chunk)}")


In [ ]:
app.get_state(config)

In [ ]:
# list(app.get_state_history(config))

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(Image(app.get_graph().draw_mermaid_png(draw_method=MermaidDrawMethod.API)))

<hr><br>

## **第4部分：为什么使用图形抽象？**

支持这种抽象非常好的一个常见论点来自传统对话建模的人，他们可能会争辩说“这就是正确的对话思维方式。”这里的确有一个论点，欢迎您思考一下。 

- **图/网络：** 每个**节点**都是处理或转换您的对话或数据的一部分的函数。**边**指定节点之间允许的流动。例如，您可能想先获取用户输入，然后进行 LLM 调用，接着是总结步骤等。

- **有限状态机（FSM）：** 从经典 FSM 的角度看，每个*不同的状态变量组合*和节点身份可以被解读为 FSM 的“状态”，每个**边**是由某些条件触发的转变。例如，下面的代码无条件地从 `[用户输入节点]` 转换到 `[LLM 响应节点]`。在更复杂的情况下，您可能会根据用户的文本进行判断。

- **马尔可夫链：** 如果您处理的是随机或基于概率的转变，可以将每个节点和后续边视为组成马尔可夫链。区别在于，在马尔可夫链中，转变通常由概率分布控制。在 LangGraph 中，您可以加入自己的分支逻辑——可能利用 LLM 的输出来决定下一步。  

> <img src="images/quizbot_state_machine.png" style="width: 1000px;"/>
> 
> <b><a href="https://ai.stanford.edu/blog/quizbot/" target="_blank">Towards an Educational Revolution Through Chatbots (2019)</a></b>

<br>

**这很好，但其中一些人可能不买账：**
- 如果您受过计算机科学培训，就会发现这只是用显式逻辑对过程依赖关系进行建模的一种方式。就像任何编程语言一样。
- 一些特别挑剔的人可能还会发现反例，即这种抽象虽然足够，但可能更好地转置，使状态成为节点，边为函数。*考虑一下文档网络或知识图谱可能需要的东西，以及为什么这种转置的设置很有趣。*

那么问题来了，如果我们已经会用 Python 编程，为什么还需要这个呢？实际上，并不是因为它尝试让您看不见编码和条件逻辑。**而是试图隐去智能体循环及其生产化的相关方面。**

<br>

#### **智能体循环（针对我们的目的）**

假设智能体的分解只是一种将一个复杂的全局函数分解为局部操作的方法：

$$F(X) = \sum_{i=0}^n (e_i \circ f_i \circ d_i)(x_i) \text{ for $i$ agents and local environments } {\bigcup_i x_i} \subseteq X$$

进一步假设，这个系统对于模拟系统随时间变化的动态尤其有用，这样我们关心的是某个未来状态 $X_T$ 通过我们对 $F(X)$ 的重复应用从某个初始状态 $X_0$ 转变而来。具体来说：

$$X_T = X_0 + \sum_{t=0}^{T-1}F(X_t) \text{ where } X_{t+1} = X_t + F(X_t)$$

然后假设 $t$ 可以在任意精细的分辨率上进行采样，您就得到了连续的智能体方程！(随意在 $[0,T]$ 区间插入积分，但不是必须的)。这就是现实世界的运作方式！

在计算机中，基于智能体的模拟也是这样结构化的，在某个抽象层次上存在一个离散时间循环。存在的问题是：
- 存在离散数量的过程，而且过程会调用其他过程。
- 对于给定的时间步，过程的数量可能非常大或非常小。通常，这只能通过观察来确定。
- 我们喜欢观察、监控、控制和版本管理（时间旅行），这需要内存和计算开销以及艰难的设计决策。
- 随着用户数量、宏观过程的复杂性以及资源的增加，过程的数量可能会大幅度扩展。

我们已经在计算机架构中重构了过程管理的问题。通常我们不需要担心这些。那么现在为什么要考虑呢？

**因为现在我们是在像 Python 这样的高抽象框架中进行处理，离散时间模拟非常低效且难以手动扩展，需要灵活性来定义我们自己的过程传播、监控、并发执行、复制和版本管理。** 事实证明，这是一个挑战，正如课程中讨论的那样。

<br>

#### **这是否意味着 LangGraph 是答案？**

LangGraph 是一个答案，而且考虑到它所支持的定制程度，入门门槛相对较低。因此，我们将继续将其作为本课程的主要抽象。它仍然有一些明显的限制，从技术上讲，还有一些门槛更高的选择，限制较少：
- **自定义图系统**可以通过一些关键抽象实现，实际上可以完全针对任何特定用例进行定制。这实际上就是 LangGraph 在后台创建和支持的内容。然而，随着您开始实现现代 LLM 应用所需的特性集，这个问题的复杂性会大幅增加，除非作为一个统一的框架发布，否则您的解决方案将本质上是定制的。
- [**NVIDIA Morpheus**](https://www.nvidia.com/en-us/ai-data-science/products/morpheus/) 是另一个有用的抽象，提供先进的数据工作流解决方案，可以用于传递推断流、跟踪分析并优化带有 CUDA 加速工作流的工作流。尽管如此，对于智能体用例，它并没有 LangGraph 所具备的所有便利性。Morpheus 和 LangGraph 的差距在复杂性上类似于 LangGraph 和 CrewAI 之间的差距，所以使用它并不是一个坏主意，但其内置的便利性会少一些。

<hr><br>

## **第五部分：** [练习] 添加简单路由

现在我们已经尽力地符合框架的期望并实现其标志性特性：**路由**。下个 Notebook 将有一个更复杂的例子，展示如何将 LangGraph 事件循环与结构化输出结合起来，实现我们在第一部分提到的个性化智能体系统。在此之前，先来做一个简单的练习，适度增强循环以构建停止条件。

- 提供了 `get_nth_message` 方法，用于从传入的状态中获取最后一条（或者是其它的）消息。
- 利用这个，试着在用户说“stop”或者 LLM 说 `stop` 时强制循环终止，任选一个就行。
- 现在流式逻辑变得更复杂了，成为一个处理更多情况的生成器（包括其他状态缓冲流和调试缓冲流）。它在 `stream_from_app_1` 中实现，您应该尝试使用它。
    - 您还可以从 `course_utils.py` 导入 `stream_from_app`（或者像这样转置 `from course_utils import stream_from_app`）。这个更好点，带一个 web 服务接口！

In [ ]:
import uuid
from typing import Annotated, Optional
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command
from langgraph.graph.message import add_messages
from functools import partial
from colorama import Fore, Style
from copy import deepcopy
import operator

##################################################################
## Define the authoritative state system (environment) for your use-case

class State(TypedDict):
    """The Graph State for your Agent System"""
    messages: Annotated[list, add_messages]
    interactions: Annotated[int, operator.__add__]
    extra_kwargs: Optional[dict]

def get_nth_message(state: State, n=-1, attr="messages"):
    try: return state.get("messages")[n].content
    except: return ""
    
##################################################################
## Define the operations (Nodes) that can happen on your environment

def user(state: State):
    """Edge" option where transition is generated at runtime"""
    answer = interrupt("[User]:")
    return {"messages": [("user", answer)]} 

def agent(state: State, config=None):
    ## Passing in config will cause connector to stream values to state modification buffer. See graph stream later
    response = llm.invoke(state.get("messages"), config=config)
    return {"messages": [response]}

def route(state: State, config=None):
    ## TODO: In the case of "stop" being found in the current state,
    ## go to the end. Otherwise, route back to the user.
    return {"interactions": 1}

##################################################################
## Define the system that organizes your nodes (and maybe edges)

builder = StateGraph(State)
builder.add_edge(START, "user")  ## A start node is always necessary
builder.add_node("agent", agent)
builder.add_node("user", user)
builder.add_node("route", route)    ## Route node declaration
builder.add_edge("user", "agent")
builder.add_edge("agent", "route")  ## Route edge declaration

##################################################################
## A memory management system to keep track of agent state
checkpointer = MemorySaver()
app = builder.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": uuid.uuid4()}}
app_stream = partial(app.stream, config=config)

##################################################################
## Simple Invocation Example

def stream_from_app_simple(app_stream, input_buffer=[{"messages": []}], verbose=False, debug=False):
    """Executes the agent system in a streaming fashion."""
    seen_metas = dict()
    input_buffer = deepcopy(input_buffer)
    
    while input_buffer:
        for mode, chunk in app_stream(input_buffer.pop(), stream_mode=["values", "messages", "updates", "debug"]):
            if mode == "messages":
                chunk, meta = chunk
                if meta.get("checkpoint_ns") not in seen_metas:
                    caller_node = meta.get("langgraph_node")
                    yield f"[{caller_node.title()}]: "
                seen_metas[meta.get("checkpoint_ns")] = meta
                if chunk.content:
                    yield chunk.content
            elif mode == "values" and verbose:
                print("[value]", chunk, flush=True)
            elif mode == "updates":
                if verbose: 
                    print("[update]", chunk, flush=True)
                if "__interrupt__" in chunk and chunk.get("__interrupt__")[0].resumable:
                    user_input = input("\n[Interrupt] " + chunk.get("__interrupt__")[0].value)
                    input_buffer.append(Command(resume=user_input))
            elif mode == "debug" and debug:
                print(f"[debug] {chunk}", flush=True)

# from course_utils import stream_from_app

import time

## We can stream over it until an interrupt is received
for token in stream_from_app_simple(app_stream, verbose=False, debug=False):
    print(token, end="", flush=True)

<br>

**注意：如果您正在使用 `course_utils` 中的 `stream_from_app`，以下接口应该是可访问的:**

In [ ]:
%%js
var url = 'http://'+window.location.host+':3002';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Trace Frontend ></h2></a>';

<br><details><summary><b>参考答案</b></summary>

```python

def route(state: State, config=None):
    ## TODO: In the case of "stop" being found in the current state,
    ## go to the end. Otherwise, route back to the user.
    if "stop" in get_nth_message(state, n=-2): 
        return {"interactions": 1}  ## Implied goto=END
    return Command(update={"interactions": 1}, goto="user")

```

</details>

<hr><br>

## **第五部分：** 反思此练习

对于刚开始构建任何类型智能体系统的您来说，LangGraph 可能会显得有些吓人。实际上，许多面临较小问题的工程师，可能完全可以依靠之前部分中的 LangChain 抽象（之前的课程对此进行了深入探讨）。与此同时，那些对智能体软件范例有深入理解的工程师，可以在原始基础和框架支持的编排之间游刃有余，并有望做出合适的设计决策，以在任何规模上推动其解决方案。

我们更喜欢把 LangGraph 当作一个很好的起始抽象，它对于一些较大的模型来说，**默认是有效的**，对于部分模型则需要**额外努力**，而对于更多可能的接口则要**进行显著修改**。*(不幸的是，Llama-8B 模型可能会落入中间或最后一类，您将在下一个 Notebook 中看到)*。话虽如此，但至少它提供了一些简单的途径进入生态系统，并且能在不太复杂的情况下扩展到生产使用场景。因此，在接下来的课程中，我们将根据需要利用此框架来支持我们的智能体循环。

- 在下一个 **练习 Notebook** 中，我们将利用 LangGraph 重新创建我们的多个个性化智能体抽象，进行下一个发言者选择和自定义状态系统，以证明该框架在某种程度上具有近乎任意的状态接口的灵活性。